In [5]:
import pandas as pd
import sqlite3
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

connection = sqlite3.connect('./database/bdfinal.sql')



In [ ]:
query = "SELECT * FROM Teams"

data = pd.read_sql(query, connection)